## Bibliotecas

In [1]:
!pip install mip

import pandas as pd
import random
import math
import time
from mip import Model, xsum, maximize, BINARY, INTEGER, OptimizationStatus
# Função para ler e processar os dados dos monitores e disciplinas a partir de um arquivo Excel.
# Ela lê o arquivo, limpa os nomes das colunas, remove duplicatas e extrai as colunas relevantes.
# Retorna um dicionário de alunos, uma lista de notas (Na) e uma matriz de disponibilidade (s_ad).
# Parâmetro:
# - path (str): Caminho para o arquivo Excel.
# Retornos:
# - alunos (dict): Dicionário onde a chave é o número USP e contem as seguintes informacoes --
#  -- > [Interesse em ser voluntario(ignorar), pontuacao do aluno (float), criterio de desempate (1, 2 ou 3), lista de materias que ele esta inscrito (Ex: [38, 39, 40]]).
# - Na (list): Lista das notas dos alunos.
# - s_ad (list of lists): Matriz binária de disponibilidade dos alunos para disciplinas.
def ler_dados(path):
    # Padroniza o nome das colunas, para facilitar extracao
    def limpar_nome_coluna(nome):
        return ' '.join(nome.strip().split()).lower()

    # Le e extrai as colunas necessarias
    df = pd.read_excel(path)
    df.columns = [limpar_nome_coluna(col) for col in df.columns]
    df = df.drop_duplicates(subset=['nºusp'], keep='last') # mantem apenas a ultima ocorrencia de um NUSP

    colunas_n = [
        'nºusp',
        'possui pedido de bolsa de estudos em andamento? (a concessão de bolsa de estudos implicará no cancelamento da monitoria a partir do início da vigência da bolsa)',
        'pretende se inscrever no peeg? (o acúmulo das duas monitorias não é permitido. caso o aluno seja selecionado nas duas modalidades precisará optar por uma delas)',
        'departamento de matemática (início da monitoria em 05/08/2024)',
        'departamento de matemática aplicada e estatística (início da monitoria em 05/08/2024)',
        'departamento de ciências de computação (início da monitoria em 05/08/2024)',
        'departamento de sistemas de computação (início da monitoria em 01/09/2024)',
        'tem interesse na monitoria voluntária (sem recebimento de bolsa)?',
        'média ponderada com reprovações'
    ]

    df_colunas = df[colunas_n].copy()

    # Calcula a pontuacao para criterio de desempate dos alunos (3 - mais prioridade, 2 - prioridade media, 1 - prioridade baixa)
    def calcular_pontuacao(row):
        bolsa = row[colunas_n[1]]
        peeg = row[colunas_n[2]]
        if bolsa == 'Sim' and peeg == 'Sim':
            return 1
        elif bolsa == 'Não' and peeg == 'Não':
            return 3
        return 2

    df_colunas['bolsa_peeg_status'] = df_colunas.apply(calcular_pontuacao, axis=1)
    df_colunas = df_colunas.drop([colunas_n[1], colunas_n[2]], axis=1)

    # Faz o mapeamento dos departamentos presentes na planilha
    departamento_mapping = {
        colunas_n[3]: 1,
        colunas_n[4]: 2,
        colunas_n[5]: 3,
        colunas_n[6]: 4
    }

    # Extrai as materias presentes na planilha
    materias_set = set()
    materias_dict = {}
    materia_index = 0

    for dept_col, dept_num in departamento_mapping.items():
        for materias in df_colunas[dept_col].dropna():
            for materia in materias.split(','):
                materia = materia.strip()
                if materia not in materias_set:
                    materias_set.add(materia)
                    materias_dict[materia_index] = (materia, dept_num)
                    materia_index += 1

    # Extrai os alunos, sua nota, sua pontuacao de desempate, se tem interesse em ser voluntario e as materias que ele se inscreveu
    alunos = df_colunas.set_index('nºusp').T.to_dict('list')

    Na = []
    for usp, valores in alunos.items():
        materias_combinadas = []
        valores_limpos = []
        for valor in valores:
            if isinstance(valor, str) and '-' in valor:
                for materia in valor.split(','):
                    materia = materia.strip()
                    for indice, (mat, dept) in materias_dict.items():
                        if materia == mat:
                            materias_combinadas.append(indice)
                            break
            elif isinstance(valor, str) and (valor.upper() == 'POS' or valor.upper() == 'PÓS'): # atribui a nota 1 caso o aluno for da pos, para diminuir sua prioridade
                valores_limpos.append(1)
            elif not pd.isna(valor):
                valores_limpos.append(valor)

        if materias_combinadas:
            valores_limpos.append(sorted(materias_combinadas))

        alunos[usp] = valores_limpos
        nota = valores_limpos[1] if len(valores_limpos) > 1 else None
        Na.append(nota)

    s_ad = [
        [1 if materia in valores_limpos[-1] else 0 for materia in materias_dict.keys()]
        for usp, valores_limpos in alunos.items()
    ]

    return alunos, Na, s_ad
# Função para criar o modelo de otimização de alocação de monitores com base nos dados fornecidos.
# Parâmetros:
# - alunos (dict): Dicionário de alunos (informações como notas, disponibilidade etc).
# - Na (list): Lista de notas dos alunos.
# - s_ad (list of lists): Matriz de disponibilidade dos alunos para disciplinas.
# - materias (list): Lista de todas as disciplinas.
# Retorno: Deve retornar o modelo de otimização, variáveis e os dados para análise posterior.
def criar_modelo(alunos, Na, s_ad, materias):
    # Voce deve implementar esta parte (foi deixado um exemplo de como pegar os parametros)
    # Esta é apenas uma sugestão de implementacao, voce pode mudar esta funcao se desejar ou ate criar o modelo
    # diretamente no codigo sem usar uma funcao (recomendo fortemente que nao faca isso)
    # Conjunto de monitores (A) e disciplinas (D)
    # A = list(range(len(alunos)))
    # D = materias
    # m = Model("Alocacao_de_Monitores", solver_name="CBC") # Pode mudar o nome do modelo se quiser, mas nao altere a parte do solver_name
    print(A)
    return m, x_ad, y_d, D

# Função para resolver o modelo de otimização com base em parâmetros fornecidos.
# Parâmetros:
# - modelo (pulp.LpProblem): O modelo de otimização que será resolvido.
# - presolve (int): Se 1, presolve é ativado; se 0, é desativado.
# - cortes (int): Intensidade da geracao de cortes (0 desativa).
# Retorno: O modelo resolvido.
def resolver_modelo(modelo, presolve, cortes):
    # Configurar as opções do solver
    modelo.preprocess = presolve  # Define preprocess (-1, 0, 1)
    modelo.cuts = cortes  # Define geração de cortes (-1, 0, 1, 2, 3)

    # Medir o tempo de execução
    start_time = time.time()

    # Resolver o modelo
    status = modelo.optimize(max_seconds=1800) # Limite de tempo de 30 minutos

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Exibir as informações do modelo resolvido
    print(f"Solução Encontrada: {modelo.objective_value}")
    print(f"Gap: {modelo.gap * 100:.2f}%")
    print(f"Tempo de execução: {execution_time:.2f} segundos")

    return modelo

# Função para exibir as informações dos monitores alocados e não alocados, além das disciplinas não atendidas.
# Parâmetros:
# - m (pulp.LpProblem): O modelo de otimização resolvido.
# - x_ad (dict): Variáveis de decisão indicando a alocação dos monitores às disciplinas.
# - y_d (dict): Variáveis de decisão indicando disciplinas não atendidas.
# - alunos (dict): Dicionário contendo as informações dos alunos.
# - D (list): Lista de disciplinas disponíveis.
# - s_ad (list of lists): Matriz de disponibilidade dos monitores para as disciplinas.
# Retorno: Nenhum. A função imprime os resultados diretamente.
def informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad):
    monitores_alocados = []
    monitores_nao_alocados = []

    # Exibe os monitores alocados para disciplinas
    for a in range(len(alunos)):
        alocado = False
        for d in D:
            if x_ad[a][d].x >= 0.99:  # Verifica se a variável é 1
                monitores_alocados.append(f"Monitor {list(alunos.keys())[a]} foi alocado para a disciplina {d}.")
                alocado = True

        # Caso o monitor não tenha sido alocado, armazenar a informação para exibir depois
        if not alocado:
            disciplinas_inscrito = [d for d in D if s_ad[a][d] == 1]
            monitores_nao_alocados.append(f"Monitor {list(alunos.keys())[a]} NÃO foi alocado. Inscrito para as disciplinas: {disciplinas_inscrito}")

    # Exibir os monitores alocados
    print("\nMonitores alocados:")
    for alocacao in monitores_alocados:
        print(alocacao)

    # Exibir os monitores não alocados
    print("\nMonitores não alocados:")
    for nao_alocacao in monitores_nao_alocados:
        print(nao_alocacao)

    # Mostrar disciplinas não atendidas e os monitores que se inscreveram para elas
    print("\nDisciplinas não atendidas e seus inscritos:")
    for d in D:
        if y_d[d].x >= 0.99:  # Verifica se a variável é 1
            monitores_inscritos = [list(alunos.keys())[a] for a in range(len(alunos)) if s_ad[a][d] == 1]
            if monitores_inscritos:
                print(f"A disciplina {d} não foi atendida por nenhum monitor. Inscritos: {monitores_inscritos}")
            else:
                print(f"A disciplina {d} não foi atendida por nenhum monitor e não possui inscritos.")


Defaulting to user installation because normal site-packages is not writeable


# Leitura dos Dados

In [2]:
# Função para ler e processar os dados dos monitores e disciplinas a partir de um arquivo Excel.
# Ela lê o arquivo, limpa os nomes das colunas, remove duplicatas e extrai as colunas relevantes.
# Retorna um dicionário de alunos, uma lista de notas (Na) e uma matriz de disponibilidade (s_ad).
# Parâmetro:
# - path (str): Caminho para o arquivo Excel.
# Retornos:
# - alunos (dict): Dicionário onde a chave é o número USP e contem as seguintes informacoes --
#  -- > [Interesse em ser voluntario(ignorar), pontuacao do aluno (float), criterio de desempate (1, 2 ou 3), lista de materias que ele esta inscrito (Ex: [38, 39, 40]]).
# - Na (list): Lista das notas dos alunos.
# - s_ad (list of lists): Matriz binária de disponibilidade dos alunos para disciplinas.
def ler_dados(path):
    # Padroniza o nome das colunas, para facilitar extracao
    def limpar_nome_coluna(nome):
        return ' '.join(nome.strip().split()).lower()

    # Le e extrai as colunas necessarias
    df = pd.read_excel(path)
    df.columns = [limpar_nome_coluna(col) for col in df.columns]
    df = df.drop_duplicates(subset=['nºusp'], keep='last') # mantem apenas a ultima ocorrencia de um NUSP

    colunas_n = [
        'nºusp',
        'possui pedido de bolsa de estudos em andamento? (a concessão de bolsa de estudos implicará no cancelamento da monitoria a partir do início da vigência da bolsa)',
        'pretende se inscrever no peeg? (o acúmulo das duas monitorias não é permitido. caso o aluno seja selecionado nas duas modalidades precisará optar por uma delas)',
        'departamento de matemática (início da monitoria em 05/08/2024)',
        'departamento de matemática aplicada e estatística (início da monitoria em 05/08/2024)',
        'departamento de ciências de computação (início da monitoria em 05/08/2024)',
        'departamento de sistemas de computação (início da monitoria em 01/09/2024)',
        'tem interesse na monitoria voluntária (sem recebimento de bolsa)?',
        'média ponderada com reprovações'
    ]

    df_colunas = df[colunas_n].copy()

    # Calcula a pontuacao para criterio de desempate dos alunos (3 - mais prioridade, 2 - prioridade media, 1 - prioridade baixa)
    def calcular_pontuacao(row):
        bolsa = row[colunas_n[1]]
        peeg = row[colunas_n[2]]
        if bolsa == 'Sim' and peeg == 'Sim':
            return 1
        elif bolsa == 'Não' and peeg == 'Não':
            return 3
        return 2

    df_colunas['bolsa_peeg_status'] = df_colunas.apply(calcular_pontuacao, axis=1)
    df_colunas = df_colunas.drop([colunas_n[1], colunas_n[2]], axis=1)

    # Faz o mapeamento dos departamentos presentes na planilha
    departamento_mapping = {
        colunas_n[3]: 1,
        colunas_n[4]: 2,
        colunas_n[5]: 3,
        colunas_n[6]: 4
    }

    # Extrai as materias presentes na planilha
    materias_set = set()
    materias_dict = {}
    materia_index = 0

    for dept_col, dept_num in departamento_mapping.items():
        for materias in df_colunas[dept_col].dropna():
            for materia in materias.split(','):
                materia = materia.strip()
                if materia not in materias_set:
                    materias_set.add(materia)
                    materias_dict[materia_index] = (materia, dept_num)
                    materia_index += 1

    # Extrai os alunos, sua nota, sua pontuacao de desempate, se tem interesse em ser voluntario e as materias que ele se inscreveu
    alunos = df_colunas.set_index('nºusp').T.to_dict('list')

    Na = []
    for usp, valores in alunos.items():
        materias_combinadas = []
        valores_limpos = []
        for valor in valores:
            if isinstance(valor, str) and '-' in valor:
                for materia in valor.split(','):
                    materia = materia.strip()
                    for indice, (mat, dept) in materias_dict.items():
                        if materia == mat:
                            materias_combinadas.append(indice)
                            break
            elif isinstance(valor, str) and (valor.upper() == 'POS' or valor.upper() == 'PÓS'): # atribui a nota 1 caso o aluno for da pos, para diminuir sua prioridade
                valores_limpos.append(1)
            elif not pd.isna(valor):
                valores_limpos.append(valor)

        if materias_combinadas:
            valores_limpos.append(sorted(materias_combinadas))

        alunos[usp] = valores_limpos
        nota = valores_limpos[1] if len(valores_limpos) > 1 else None
        Na.append(nota)

    s_ad = [
        [1 if materia in valores_limpos[-1] else 0 for materia in materias_dict.keys()]
        for usp, valores_limpos in alunos.items()
    ]

    return alunos, Na, s_ad

# Modelo

In [3]:
# Função para criar o modelo de otimização de alocação de monitores com base nos dados fornecidos.
# Parâmetros:
# - alunos (dict): Dicionário de alunos (informações como notas, disponibilidade etc).
# - Na (list): Lista de notas dos alunos.
# - s_ad (list of lists): Matriz de disponibilidade dos alunos para disciplinas.
# - materias (list): Lista de todas as disciplinas.
# Retorno: Deve retornar o modelo de otimização, variáveis e os dados para análise posterior.
def criar_modelo(alunos, Na, s_ad, materias):
    # Voce deve implementar esta parte (foi deixado um exemplo de como pegar os parametros)
    # Esta é apenas uma sugestão de implementacao, voce pode mudar esta funcao se desejar ou ate criar o modelo
    # diretamente no codigo sem usar uma funcao (recomendo fortemente que nao faca isso)
    # Conjunto de monitores (A) e disciplinas (D)
    # A = list(range(len(alunos)))
    # D = materias
    # m = Model("Alocacao_de_Monitores", solver_name="CBC") # Pode mudar o nome do modelo se quiser, mas nao altere a parte do solver_name
    print(A)
    return m, x_ad, y_d, D

# Função para resolver o modelo de otimização com base em parâmetros fornecidos.
# Parâmetros:
# - modelo (pulp.LpProblem): O modelo de otimização que será resolvido.
# - presolve (int): Se 1, presolve é ativado; se 0, é desativado.
# - cortes (int): Intensidade da geracao de cortes (0 desativa).
# Retorno: O modelo resolvido.
def resolver_modelo(modelo, presolve, cortes):
    # Configurar as opções do solver
    modelo.preprocess = presolve  # Define preprocess (-1, 0, 1)
    modelo.cuts = cortes  # Define geração de cortes (-1, 0, 1, 2, 3)

    # Medir o tempo de execução
    start_time = time.time()

    # Resolver o modelo
    status = modelo.optimize(max_seconds=1800) # Limite de tempo de 30 minutos

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Exibir as informações do modelo resolvido
    print(f"Solução Encontrada: {modelo.objective_value}")
    print(f"Gap: {modelo.gap * 100:.2f}%")
    print(f"Tempo de execução: {execution_time:.2f} segundos")

    return modelo

# Função para exibir as informações dos monitores alocados e não alocados, além das disciplinas não atendidas.
# Parâmetros:
# - m (pulp.LpProblem): O modelo de otimização resolvido.
# - x_ad (dict): Variáveis de decisão indicando a alocação dos monitores às disciplinas.
# - y_d (dict): Variáveis de decisão indicando disciplinas não atendidas.
# - alunos (dict): Dicionário contendo as informações dos alunos.
# - D (list): Lista de disciplinas disponíveis.
# - s_ad (list of lists): Matriz de disponibilidade dos monitores para as disciplinas.
# Retorno: Nenhum. A função imprime os resultados diretamente.
def informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad):
    monitores_alocados = []
    monitores_nao_alocados = []

    # Exibe os monitores alocados para disciplinas
    for a in range(len(alunos)):
        alocado = False
        for d in D:
            if x_ad[a][d].x >= 0.99:  # Verifica se a variável é 1
                monitores_alocados.append(f"Monitor {list(alunos.keys())[a]} foi alocado para a disciplina {d}.")
                alocado = True

        # Caso o monitor não tenha sido alocado, armazenar a informação para exibir depois
        if not alocado:
            disciplinas_inscrito = [d for d in D if s_ad[a][d] == 1]
            monitores_nao_alocados.append(f"Monitor {list(alunos.keys())[a]} NÃO foi alocado. Inscrito para as disciplinas: {disciplinas_inscrito}")

    # Exibir os monitores alocados
    print("\nMonitores alocados:")
    for alocacao in monitores_alocados:
        print(alocacao)

    # Exibir os monitores não alocados
    print("\nMonitores não alocados:")
    for nao_alocacao in monitores_nao_alocados:
        print(nao_alocacao)

    # Mostrar disciplinas não atendidas e os monitores que se inscreveram para elas
    print("\nDisciplinas não atendidas e seus inscritos:")
    for d in D:
        if y_d[d].x >= 0.99:  # Verifica se a variável é 1
            monitores_inscritos = [list(alunos.keys())[a] for a in range(len(alunos)) if s_ad[a][d] == 1]
            if monitores_inscritos:
                print(f"A disciplina {d} não foi atendida por nenhum monitor. Inscritos: {monitores_inscritos}")
            else:
                print(f"A disciplina {d} não foi atendida por nenhum monitor e não possui inscritos.")


# Gerar Instâncias Maiores

# Exemplo de Uso

In [5]:
# Caminho do arquivo Excel com os dados originais
path = 'Dados_monitores.xlsx'

# Aumenta os dados de alunos e matérias
alunos, Na, s_ad, materias = aumentar_dados(path, escala=1)

# Cria o modelo de otimização com os dados aumentados
mod, x_ad, y_d, D = criar_modelo(alunos, Na, s_ad, materias)

# Resolve o modelo
m = resolver_modelo(mod, 1, 1)

# informacoes_monitores(m, x_ad, y_d, alunos, D, s_ad)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.